In [4]:
%%writefile main.py
#!/usr/bin/env python3
import sys
import subprocess
import numpy as np
import pandas as pd
##pip3 install pyslha
#import pyslha # not longer required
import tempfile
import os
import re
def grep(pattern,multilinestring):
    '''Grep replacement in python
    as in: $ echo $multilinestring | grep pattern
    dev: re.M is for multiline strings
    '''
    import re 
    grp=re.finditer('(.*)%s(.*)' %pattern, multilinestring,re.M)
    return '\n'.join([g.group(0) for g in grp])

def subprocess_line_by_line(*args,TRUST_ERRORS=True,**kwargs):
    '''
    Subprocess output line by line. Stop of error found when TRUST_ERRORS=True, and simply
    report wait method otherwise.
    
    The arguments are the same as for the Popen constructor.
    
    WARNING: Works only in Python 3
    
    See: https://stackoverflow.com/a/28319191/2268280 
    and: https://stackoverflow.com/a/17698359/2268280
    
    Example:
    
    subprocess_line_by_line('for i in $(seq 1 3);do echo $i; sleep 1;done',shell=True)
    '''
    
    if not TRUST_ERRORS:
        kwargs['stderr']=subprocess.PIPE
        
    kwargs['stdout']=subprocess.PIPE
    kwargs['bufsize']=1
    kwargs['universal_newlines']=True
    s=subprocess.Popen(*args,**kwargs)
    with s as p:
        for line in p.stdout:
            print(line, end='') # process line here
    
    if TRUST_ERRORS:
        if p.returncode != 0:
            raise subprocess.CalledProcessError(p.returncode, p.args)
    else:
        return s.wait()
    
##Main madgraph script
def preamble(MHc=750,MH0=110):
    return '''import model InertDoublet_UFO
define p = g u c d s b u~ c~ d~ s~ b~
define j = p  
define l+ = e+ mu+ 
define l- = e- mu- 
define vl = ve vm vt 
define vl~ = ve~ vm~ vt~

generate p p > h2 h2 j j @0

output ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL

'''

def lamL_loop(LHA_input_file,MHc=750,MH0=110,lamL=0.01):
    return '''launch ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL
0    
../'''+LHA_input_file+'''
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL '''+str(lamL)+'''
set mmh0 '''+str(MH0)+'''
set mma0 '''+str(MHc)+'''
set mmhch '''+str(MHc)+'''
0

''' 


def closing(MHc=750,MH0=110):
    return '''launch ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL -i
print_results --path=./result_BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lamdaL_mh0_110.txt --format=short


done
'''

thisroot='/home/restrepo/prog/ROOT/root/bin/thisroot.sh'
#thisroot='/opt/root5/bin/thisroot.sh'
MHc=750
MH0=260
if MH0%1!=0:
    sys.exit('ERROR: MH0 must be integer')
MADGRAPH='madgraph'
run_dir='Task_Asana'
work_dir='studies/IDM/'+run_dir
work_script='BP_'+str(int(MHc))+'_A_'+str(int(MH0))+'.txt'
LHA_input_file='MadGraph_cards/benchmarks/param_card_template.dat'
output_dir='studies/IDM/BP_'+str(int(MHc))+'_'+str(int(MH0))+'_vs_lambdaL'

CLONE_GIT_REPO=True #WARNING: Try to overwrite current contents!
INSTALL=True # If True check full installation
TEST=False #take a long time
VERBOSE=True #Print shell commands output line by line
if not INSTALL:
    CLONE_GIT_REPO=False

f=open('kk.sh','w')
f.write('source '+thisroot+'\n')
f.write('which root\n')
f.close()

if not subprocess.Popen('bash kk.sh'.split(),
            stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()[0]:
    sys.exit('INSTALL ROOT: see instrucctions in notebook')

print("============== VA 1: CLONE GIT ===========")
if CLONE_GIT_REPO:  
    REPO='VBF_IDM'
    REPO_url='git@github.com:restrepo'
    main_dir='.' #WARNING: Try to overwirte contents
    if os.path.exists(main_dir+'index.ipynb'):
        sys.exit('ERROR: Repo files already exists. Check main_dir')
    if not os.path.isdir(main_dir):
        s=subprocess.Popen(['mkdir','-p',main_dir],stdout=subprocess.PIPE,stderr=subprocess.PIPE).wait()

    td=tempfile.mkdtemp()
    s=subprocess_line_by_line(('git clone  --recursive '+REPO_url+'/'+REPO+'.git').split(),cwd=td,
                 stdout=subprocess.PIPE,stderr=subprocess.PIPE,TRUST_ERRORS=False)

    s=subprocess.Popen('mv '+td+'/'+REPO+'/*  '+main_dir,shell=True,
                   stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

    s=subprocess.Popen('mv '+td+'/'+REPO+'/.* '+main_dir,shell=True,
                   stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

    os.rmdir(td+'/'+REPO)
    os.rmdir(td)
    
if INSTALL:
    s=subprocess.Popen('git checkout -b v2.3.3'.split(),cwd='madgraph',
                      stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()
    if 'Switched' not in s[1].decode('utf-8'):
        sys.exit('Submodule problems')

f=open('madgraph/kk.sh','w')
f.write('source '+thisroot+'\n')
f.write('./bin/mg5_aMC install.dat\n')
f.close()

print("============== VA 2: INSTALL ===========")

if INSTALL:
    if VERBOSE:
        subprocess_line_by_line('bash kk.sh'.split(),cwd='madgraph', TRUST_ERRORS=False )
    else:
        s=subprocess.call('bash kk.sh'.split(),cwd='madgraph', stdout=open('kk','w'),stderr=open('kkk','w') )

if TEST:
    if VERBOSE:
        subprocess_line_by_line('./test.sh'.split(), cwd='test',
            stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    else:
    s=subprocess.Popen('./test.sh'.split(), cwd='test',
            stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

LambdasL=[0.01,0.02,0.05,0.07,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,1.0,3.0,5.0,7.0,10.0]

## MadGRAPH
s=subprocess.Popen(['mkdir','-p',work_dir],
                 stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()



f=open(work_dir+'/'+work_script,'w')
f.write( preamble(MHc,MH0) )
for lamL in LambdasL:
    f.write( lamL_loop(LHA_input_file,MHc,MH0,lamL=lamL) )
f.write( closing(MHc,MH0) )
f.close()

f=open(MADGRAPH+'/kk.sh','w')
f.write('source '+thisroot+'\n')
f.write('./bin/mg5_aMC ../'+work_dir+'/'+work_script+'\n')
f.close()

print("****VA 3: Run Madgraph*****")

if not VERBOSE:
    s=subprocess.Popen( 'bash kk.sh'.split(), cwd=MADGRAPH,
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    s.wait()

if VERBOSE:
    subprocess_line_by_line( 'bash kk.sh'.split(), cwd=MADGRAPH,TRUST_ERRORS=False)

## MadEvent
print("****VA 4: Run MadEvent*****")

pythia_script='TemplateRunPythiaDelphes_all.dat'
if len(LambdasL)>99:
    sys.exit('ERROR: UPDATE FORMAT FOR > 99 runs')
f=open(work_dir+'/'+pythia_script,'w')
for r in range(1,len(LambdasL)+1):
    f.write('pythia run_%02d\n' %r)
    f.write('3\n')
    f.write('0\n')
f.close()

f=open(output_dir+'/kk.sh','w')
f.write('source '+thisroot+'\n')
f.write('./bin/madevent ../'+run_dir+'/'+pythia_script+'\n')
f.close()

#if not VERBOSE:
s=subprocess.Popen('bash kk.sh'.split(), cwd=output_dir,
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)

s.wait()
(PHOUT,PHERR)=s.communicate()

cs_pb=np.array( re.sub( '\s+\+\-\s+[0-9\+\-eE\.]+\s+pb','\n',  
          re.sub('\s+Cross-section\s+:\s+','' ,
          ''.join( grep('Cross-section',PHOUT.decode('utf-8')).split('\n') 
          ) )  ).strip().split('\n')  ).astype(float64)

if len(cs_pb)==len(LambdasL):
    df=pd.DataFrame({'xs_'+str(int(MH0)):cs_pb,'laL':LambdasL})
else:
    sys.exit('Error: missing cross section')

df.to_csv('output/cs_'+str(int(MHc))+'_'+str(int(MH0))+'.csv',index=False)

## Results
full_output='output'
s=subprocess.Popen(['mkdir', '-p',full_output]).wait()

#if 'xs_'+str(int(MH0)) not in df.columns.values:
if True:
    for r in range(1,len(LambdasL)+1):
        nrun='%02d' %r
        nrun3='%03d' %r
        s=subprocess.Popen(['cp',output_dir+'/Events/run_'+nrun+'/tag_1_delphes_events.root', 
                            full_output+'/delphes_events_'+str(int(MH0))+'_'+nrun3+'_.root'],
                             stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if s.wait()>0:
            sys.exit('Files not found')

Overwriting main.py
